# HackerEarth ML 1:  Predict the Road Sign
### XGBoost solution: 98%

Download the dataset from the following link.

[Dataset Download](https://he-s3.s3.amazonaws.com/media/hackathon/hackerearth/predict-the-road-sign/4b699168-4-here_dataset.zip)


In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from xgboost import XGBClassifier as XGBC
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

Defining folder paths and reading train and test data.

In [2]:
model_name = "XGBC"
obj_dir = "./obj"
data_dir = "./data"
output_dir = "./output"

train_file = os.path.join(data_dir, "train.csv")
test_file = os.path.join(data_dir, "test.csv")
op_file = os.path.join(output_dir, model_name+"_"+datetime.now().strftime('%Y%m%d%H%M%S')+".csv")
model_file = os.path.join(obj_dir, model_name+"_"+datetime.now().strftime('%Y%m%d')+".pkl")

print("Output file: {}\nModel file: {}".format(op_file, model_file))

Output file: ./output/XGBC_20180901193012.csv
Model file: ./obj/XGBC_20180901.pkl


In [3]:
train = pd.read_csv(train_file)
print("Train data info: ", train.info())

test = pd.read_csv(test_file)
print("Test data info: ", test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38485 entries, 0 to 38484
Data columns (total 7 columns):
Id                     38485 non-null object
DetectedCamera         38485 non-null object
AngleOfSign            38485 non-null int64
SignAspectRatio        38485 non-null float64
SignWidth              38485 non-null int64
SignHeight             38485 non-null int64
SignFacing (Target)    38485 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 2.1+ MB
Train data info:  None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31485 entries, 0 to 31484
Data columns (total 6 columns):
Id                 31485 non-null object
DetectedCamera     31485 non-null object
AngleOfSign        31485 non-null int64
SignAspectRatio    31485 non-null float64
SignWidth          31485 non-null int64
SignHeight         31485 non-null int64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.4+ MB
Test data info:  None


Create a common mapping dictionary for the camera and target columns. Apply the mapping and dropping Aspect Ratio feature.

In [4]:
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}

train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})
train = train.replace({'SignFacing (Target)':mapping})

y_train = train['SignFacing (Target)']
test_id = test['Id']

df = train.append(test, sort=False)
print(df.shape, train.shape, test.shape)

drop_cols = ['SignAspectRatio', 'SignFacing (Target)', 'Id']
df.drop(columns=drop_cols, inplace=True)
X_train = df.iloc[:len(train), :]
print(X_train.shape)
X_test = df.iloc[len(train):, :]
print(X_test.shape)

(69970, 7) (38485, 7) (31485, 6)
(38485, 6)
(31485, 6)


Building the random forest classifier model. Model file will be saved to the obj directory.

In [5]:
xgbc = XGBC(max_depth=3, learning_rate=0.01, n_estimators=300, silent=True, objective='multi:softprob', booster='gbtree', n_jobs=1, nthread=3, seed=10, eval_metric='mlogloss')
xgbc.fit(X_train, y_train)
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.02, 0.03, 0.05],
    'n_estimators': [300, 350, 400]
}
joblib.dump(xgbc, "./obj/xgbc_base.pkl")
grid_search = GridSearchCV(estimator = xgbc, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train);

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] learning_rate=0.01, max_depth=3, n_estimators=300 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=300 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=300 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=300 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=300 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=350 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=350 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=350 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=350 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=350 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=400 ...............
[CV] learning_rate=0.01, max_depth=3, n_estimators=400 ...............


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

[CV]  learning_rate=0.01, max_depth=3, n_estimators=300, total=  11.8s
[CV] learning_rate=0.01, max_depth=3, n_estimators=400 ...............


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  learning_rate=0.01, max_depth=3, n_estimators=300, total=  11.9s
[CV] learning_rate=0.01, max_depth=3, n_estimators=400 ...............


...

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  learning_rate=0.05, max_depth=5, n_estimators=400, total=  11.7s


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  learning_rate=0.05, max_depth=5, n_estimators=400, total=  10.1s


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  4.5min finished


Predicting the target probabilities for the test dataset.

In [6]:
print(grid_search.best_params_)
best_grid = grid_search.best_estimator_
joblib.dump(xgbc, model_file)
y_test = grid_search.predict_proba(X_test)

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}


Saving the submission file.

In [7]:
submission = pd.DataFrame(data=y_test, columns=['Front','Left','Rear','Right'])
submission['Id'] = test_id
submission = submission[['Id','Front','Left','Rear','Right']]
submission.to_csv(op_file, index=False)